In [2]:
import re
import numpy as np
import pandas as pd
from scipy import stats
from pathlib import Path
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from pandas.api.types import CategoricalDtype
from scipy.stats import chi2, norm

**Calculate Mean and StD**

In [6]:
res = []

mean_res = np.mean(res)
std_res = np.std(res)

print(mean_res)
print(std_res)

nan
nan


/home/hahlers/miniconda3/envs/server_env/lib/python3.10/site-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/hahlers/miniconda3/envs/server_env/lib/python3.10/site-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/hahlers/miniconda3/envs/server_env/lib/python3.10/site-packages/numpy/_core/_methods.py:218: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/hahlers/miniconda3/envs/server_env/lib/python3.10/site-packages/numpy/_core/_methods.py:175: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/home/hahlers/miniconda3/envs/server_env/lib/python3.10/site-packages/numpy/_core/_methods.py:210: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


**Get Mean Accuracies for different setups**

In [4]:
path = "pacs_tta_results_all_domains_20250815_123038.txt"

with open(path, encoding="utf-8", errors="ignore") as f:
    lines = f.readlines()

records = []
current_domain, current_mode, current_seed, current_target = None, None, None, None
in_seed_block = False

re_domain   = re.compile(r"^=== TEST DOMAIN:\s*([A-Z_]+)\s*===")
re_mode     = re.compile(r"^Mode:\s*([A-Z_0-9]+)\s*$")
re_seed     = re.compile(r"^Seed:\s*(\d+)\s*$")
re_target   = re.compile(r"^\s*Target\s+([a-zA-Z_]+):\s*$")
re_accuracy = re.compile(r"^\s*Accuracy:\s*([0-9]*\.[0-9]+)\s*$")

def norm_domain(d): return d.lower().replace("_", " ")
def norm_mode(m): return m.lower()

for line in lines:
    m = re_domain.match(line)
    if m:
        current_domain = norm_domain(m.group(1))
        continue
    m = re_mode.match(line)
    if m:
        current_mode = norm_mode(m.group(1))
        continue
    m = re_seed.match(line)
    if m:
        current_seed = int(m.group(1))
        in_seed_block = True
        continue
    if in_seed_block:
        mt = re_target.match(line)
        if mt:
            current_target = mt.group(1).lower()
            continue
        ma = re_accuracy.match(line)
        if ma and current_target is not None:
            acc = float(ma.group(1))
            records.append({
                "domain": current_domain,
                "mode": current_mode,
                "seed": current_seed,
                "target": current_target,
                "accuracy": acc,
            })

df = pd.DataFrame(records)

def mean_excl_original(group):
    vals = [v for t, v in zip(group["target"], group["accuracy"]) if t != "original"]
    return sum(vals)/len(vals) if vals else float("nan")

agg = (
    df.groupby(["domain","mode","seed"])
      .apply(lambda g: mean_excl_original(g.reset_index(drop=True)))
      .reset_index(name="mean_accuracy")
)

for _, row in agg.iterrows():
    print(f"{row['domain']}, {row['mode']}, seed {row['seed']}: {row['mean_accuracy']:.4f}")


art painting, average, seed 0: 0.1245
art painting, average, seed 7: 0.1851
art painting, average, seed 42: 0.1851
art painting, selective_0_1, seed 0: 0.8625
art painting, selective_0_1, seed 7: 0.8612
art painting, selective_0_1, seed 42: 0.8846
art painting, selective_0_2, seed 0: 0.8190
art painting, selective_0_2, seed 7: 0.8314
art painting, selective_0_2, seed 42: 0.8530
art painting, selective_0_3, seed 0: 0.1473
art painting, selective_0_3, seed 7: 0.0953
art painting, selective_0_3, seed 42: 0.1626
art painting, selective_1_2, seed 0: 0.8197
art painting, selective_1_2, seed 7: 0.8190
art painting, selective_1_2, seed 42: 0.8446
art painting, selective_1_3, seed 0: 0.1473
art painting, selective_1_3, seed 7: 0.0953
art painting, selective_1_3, seed 42: 0.1626
art painting, selective_2_3, seed 0: 0.1473
art painting, selective_2_3, seed 7: 0.0953
art painting, selective_2_3, seed 42: 0.1626
art painting, single_0, seed 0: 0.8651
art painting, single_0, seed 7: 0.8613
art paint

/tmp/ipykernel_3178060/595327797.py:57: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: mean_excl_original(g.reset_index(drop=True)))


In [7]:
FILE = Path("pacs_tta_results_all_domains_20250815_123038.txt")

# %% Parser
test_pat  = re.compile(r"^=== TEST DOMAIN:\s+([A-Z_]+)\s+===")
mode_pat  = re.compile(r"^Mode:\s+([A-Z0-9_]+)")
seed_pat  = re.compile(r"^Seed:\s+(\d+)")
tgt_pat   = re.compile(r"^\s*Target\s+([a-z_]+):\s*$", re.IGNORECASE)
acc_pat   = re.compile(r"^\s*Accuracy:\s*([0-9]*\.?[0-9]+)\s*$")

def normalize_domain(name):
    # Eingabe kann z.B. 'ART_PAINTING' (TEST) oder 'art_painting' (Target) sein
    name = name.strip()
    name = name.lower()
    name = name.replace(" ", "_")
    return name

rows = []  # rohe Messungen: dict(test_domain, mode, seed, target_domain, acc)

cur_test = None
cur_mode = None
cur_seed = None
cur_target = None

with FILE.open("r", encoding="utf-8") as f:
    for line in f:
        line = line.rstrip("\n")

        m = test_pat.match(line)
        if m:
            cur_test = normalize_domain(m.group(1))
            cur_mode = None
            cur_seed = None
            cur_target = None
            continue

        m = mode_pat.match(line)
        if m:
            cur_mode = m.group(1).upper()  # z.B. SINGLE_0, SELECTIVE_0_1, AVERAGE
            cur_seed = None
            cur_target = None
            continue

        m = seed_pat.match(line)
        if m:
            cur_seed = int(m.group(1))
            cur_target = None
            continue

        m = tgt_pat.match(line)
        if m:
            cur_target = normalize_domain(m.group(1))
            continue

        m = acc_pat.match(line)
        if m and cur_test and cur_mode and cur_seed is not None and cur_target:
            acc = float(m.group(1))
            rows.append({
                "test_domain": cur_test,
                "mode": cur_mode,
                "seed": cur_seed,
                "target_domain": cur_target,
                "accuracy": acc
            })
            # nach Accuracy-Zeile lassen wir cur_target gesetzt, falls weitere Felder folgen (Ignorieren ok)
            continue

# In DataFrame laden
df = pd.DataFrame(rows)

# Nur echte Target-Domains (ohne "original")
df_no_orig = df[df["target_domain"] != "original"].copy()

# Aggregation: mean & std je (test_domain, mode, target_domain)
agg = (
    df_no_orig
    .groupby(["test_domain", "mode", "target_domain"], as_index=False)
    .agg(mean_acc=("accuracy", "mean"),
         std_acc =("accuracy", "std"),
         n_seeds=("accuracy", "size"))
)

# Ordnung der Domains optional konsistent halten
domain_order = ["art_painting", "cartoon", "photo", "sketch"]
agg["test_domain"] = pd.Categorical(agg["test_domain"], domain_order, ordered=True)
agg["target_domain"] = pd.Categorical(agg["target_domain"], domain_order, ordered=True)
agg = agg.sort_values(["test_domain", "mode", "target_domain"]).reset_index(drop=True)

# "Tidy" Tabelle ausgeben
print("Tidy-Tabelle (eine Zeile je test_domain / mode / target_domain):")
display(agg)

# Hübsche Pivot-Tabelle: Zeilen = (test_domain, mode), Spalten = target_domain, Werte = "mean ± std"
def fmt(mean, std):
    if pd.isna(std):
        return f"{mean:.4f} ± nan"
    return f"{mean:.4f} ± {std:.4f}"

pivot = (
    agg
    .assign(mean_std=agg.apply(lambda r: fmt(r["mean_acc"], r["std_acc"]), axis=1))
    .pivot_table(index=["test_domain", "mode"], columns="target_domain", values="mean_std", aggfunc="first")
    .reindex(columns=[d for d in domain_order if d in agg["target_domain"].unique()])
    .sort_index()
)

print("\nPivot (mean ± std):")
display(pivot)

Tidy-Tabelle (eine Zeile je test_domain / mode / target_domain):


,test_domain,mode,target_domain,mean_acc,std_acc,n_seeds
0,art_painting,AVERAGE,cartoon,0.164900,0.034987,3
1,art_painting,AVERAGE,photo,0.164900,0.034987,3
2,art_painting,AVERAGE,sketch,0.164900,0.034987,3
3,art_painting,SELECTIVE_0_1,cartoon,0.875000,0.006800,3
4,art_painting,SELECTIVE_0_1,photo,0.867167,0.013679,3
...,...,...,...,...,...,...
127,sketch,SINGLE_2,cartoon,0.696867,0.047704,3
128,sketch,SINGLE_2,photo,0.711633,0.060170,3
129,sketch,SINGLE_3,art_painting,0.142967,0.088598,3
130,sketch,SINGLE_3,cartoon,0.144567,0.089951,3



Pivot (mean ± std):


/tmp/ipykernel_3163841/18926079.py:99: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  agg


target_domain                  art_painting          cartoon            photo  \
test_domain  mode                                                               
art_painting AVERAGE                    NaN  0.1649 ± 0.0350  0.1649 ± 0.0350   
             SELECTIVE_0_1              NaN  0.8750 ± 0.0068  0.8672 ± 0.0137   
             SELECTIVE_0_2              NaN  0.8520 ± 0.0120  0.8254 ± 0.0327   
             SELECTIVE_0_3              NaN  0.1134 ± 0.0265  0.1788 ± 0.0699   
             SELECTIVE_1_2              NaN  0.8478 ± 0.0096  0.8153 ± 0.0352   
             SELECTIVE_1_3              NaN  0.1134 ± 0.0265  0.1788 ± 0.0699   
             SELECTIVE_2_3              NaN  0.1134 ± 0.0265  0.1788 ± 0.0699   
             SINGLE_0                   NaN  0.8688 ± 0.0039  0.8662 ± 0.0104   
             SINGLE_1                   NaN  0.8814 ± 0.0051  0.8687 ± 0.0091   
             SINGLE_2                   NaN  0.8420 ± 0.0117  0.8244 ± 0.0281   
             SINGLE_3                   NaN  0.1134 ± 0.0265  0.1788 ± 0.0699   
cartoon      AVERAGE        0.1660 ± 0.0000              NaN  0.1660 ± 0.0000   
             SELECTIVE_0_1  0.7710 ± 0.0030              NaN  0.7794 ± 0.0177   
             SELECTIVE_0_2  0.7857 ± 0.0084              NaN  0.8063 ± 0.0108   
             SELECTIVE_0_3  0.1802 ± 0.0128              NaN  0.1446 ± 0.0256   
             SELECTIVE_1_2  0.7856 ± 0.0106              NaN  0.8028 ± 0.0171   
             SELECTIVE_1_3  0.1802 ± 0.0128              NaN  0.1446 ± 0.0256   
             SELECTIVE_2_3  0.1802 ± 0.0128              NaN  0.1446 ± 0.0256   
             SINGLE_0       0.7435 ± 0.0162              NaN  0.7614 ± 0.0165   
             SINGLE_1       0.7813 ± 0.0090              NaN  0.7895 ± 0.0132   
             SINGLE_2       0.7812 ± 0.0133              NaN  0.8013 ± 0.0062   
             SINGLE_3       0.1802 ± 0.0128              NaN  0.1446 ± 0.0256   
photo        AVERAGE        0.1144 ± 0.0061  0.1118 ± 0.0024              NaN   
             SELECTIVE_0_1  0.9707 ± 0.0053  0.9675 ± 0.0019              NaN   
             SELECTIVE_0_2  0.9587 ± 0.0118  0.9529 ± 0.0100              NaN   
             SELECTIVE_0_3  0.1597 ± 0.0857  0.1819 ± 0.0708              NaN   
             SELECTIVE_1_2  0.9497 ± 0.0213  0.9501 ± 0.0075              NaN   
             SELECTIVE_1_3  0.1597 ± 0.0857  0.1819 ± 0.0708              NaN   
             SELECTIVE_2_3  0.1597 ± 0.0857  0.1819 ± 0.0708              NaN   
             SINGLE_0       0.9755 ± 0.0039  0.9719 ± 0.0033              NaN   
             SINGLE_1       0.9725 ± 0.0039  0.9679 ± 0.0058              NaN   
             SINGLE_2       0.9601 ± 0.0137  0.9575 ± 0.0048              NaN   
             SINGLE_3       0.1597 ± 0.0857  0.1819 ± 0.0708              NaN   
sketch       AVERAGE        0.0926 ± 0.0900  0.0926 ± 0.0900  0.0926 ± 0.0900   
             SELECTIVE_0_1  0.7681 ± 0.0432  0.7732 ± 0.0268  0.7735 ± 0.0516   
             SELECTIVE_0_2  0.7430 ± 0.0222  0.7097 ± 0.0419  0.7326 ± 0.0349   
             SELECTIVE_0_3  0.1430 ± 0.0886  0.1446 ± 0.0900  0.0719 ± 0.0724   
             SELECTIVE_1_2  0.7557 ± 0.0411  0.7115 ± 0.0551  0.7284 ± 0.0633   
             SELECTIVE_1_3  0.1430 ± 0.0886  0.1446 ± 0.0900  0.0719 ± 0.0724   
             SELECTIVE_2_3  0.1430 ± 0.0886  0.1446 ± 0.0900  0.0719 ± 0.0724   
             SINGLE_0       0.7173 ± 0.0706  0.7298 ± 0.0610  0.7350 ± 0.0522   
             SINGLE_1       0.7819 ± 0.0321  0.7918 ± 0.0225  0.7902 ± 0.0315   
             SINGLE_2       0.7361 ± 0.0455  0.6969 ± 0.0477  0.7116 ± 0.0602   
             SINGLE_3       0.1430 ± 0.0886  0.1446 ± 0.0900  0.0719 ± 0.0724   

target_domain                        sketch  
test_domain  mode                            
art_painting AVERAGE        0.1649 ± 0.0350  
             SELECTIVE_0_1  0.8661 ± 0.0203  
             SELECTIVE_0_2  0.8260 ± 0.0296  
             SELECTIVE_0_3  0.1129 ± 0.0200

**EXPERIMENT: Linear Mixed Model for Statistical Evaluation**

In [8]:
df = pd.read_csv("pacs_results.csv")

df = df.rename(columns={
    "domain": "domain",
    "seed": "seed",
    "approach": "approach",
    "mode": "mode",
    "mean accuracy": "accuracy"
})
df["accuracy"] = pd.to_numeric(df["accuracy"], errors="coerce")
df = df.dropna(subset=["accuracy", "domain", "seed", "approach", "mode"]).copy()

# categories
df["approach"] = pd.Categorical(df["approach"], categories=["MixStyle", "TTA"])
df.loc[df["approach"] == "MixStyle", "mode"] = "base"
df["mode"] = pd.Categorical(df["mode"])

# domain × seed id for random effects
df["domain_seed"] = df["domain"].astype(str) + "_" + df["seed"].astype(str)

# ----------------------
# 1) TTA-only
# ----------------------
df_tta = df[df["approach"] == "TTA"].copy()
df_tta["mode"] = df_tta["mode"].cat.remove_unused_categories()

vc_tta = {"seed": "0 + C(domain_seed)"}
model_tta = smf.mixedlm(
    "accuracy ~ C(mode)",
    data=df_tta,
    groups=df_tta["domain"],
    vc_formula=vc_tta
)
res_tta = model_tta.fit(method="lbfgs", reml=False, maxiter=200)
print("\n=== TTA-only: Mode-Effekte ===")
print(res_tta.summary())

# ----------------------
# 2) MixStyle (base) vs TTA modes
# ----------------------
best_modes = ["single_0", "single_1", "single_2", "selective_0_1", "selective_0_2", "selective_1_2"]

df_compare = df[
    ((df["approach"] == "MixStyle") & (df["mode"] == "base"))
    | ((df["approach"] == "TTA") & (df["mode"].isin(best_modes)))
].copy()

df_compare["mode"] = pd.Categorical(
    df_compare["mode"],
    categories=["base"] + [m for m in best_modes if m in df_compare["mode"].unique()]
)

vc_compare = {"seed": "0 + C(domain_seed)"}

model_compare = smf.mixedlm(
    "accuracy ~ C(mode, Treatment(reference='base'))",
    data=df_compare,
    groups=df_compare["domain"],
    vc_formula=vc_compare
)

try:
    res_compare = model_compare.fit(method="lbfgs", reml=False, maxiter=200)
except Exception:
    model_compare_simple = smf.mixedlm(
        "accuracy ~ C(mode, Treatment(reference='base'))",
        data=df_compare,
        groups=df_compare["domain"]
    )
    res_compare = model_compare_simple.fit(method="lbfgs", reml=False, maxiter=200)

print("\n=== MixStyle (base) vs. einzelne TTA-Modi ===")
print(res_compare.summary())

# ----------------------
# 3) Effekt-Tabelle extrahieren
# ----------------------
def fixed_effects_table(res):
    ci = res.conf_int()
    return pd.DataFrame({
        "coef": res.params,
        "ci_low": ci[0],
        "ci_high": ci[1]
    })

print("\nFixed Effects (TTA-only):\n", fixed_effects_table(res_tta))
print("\nFixed Effects (MixStyle vs. beste TTA):\n", fixed_effects_table(res_compare))

# ----------------------
# 4) MixStyle (base) vs domain x mode
# ----------------------

modes_available = [m for m in df_compare["mode"].cat.categories if m != "base"]

# Wide-Table: Zeilen = (domain, seed), Spalten = mode, Werte = accuracy
wide = (
    df_compare.pivot_table(index=["domain", "seed"], columns="mode", values="accuracy", aggfunc="mean")
    .reindex(columns=["base"] + modes_available)
)

results = []
for dom in wide.index.get_level_values("domain").unique():
    sub = wide.loc[dom]  # DataFrame: index=seed, cols=base + modes
    for mode in modes_available:
        diffs = (sub[mode] - sub["base"]).dropna()  # nur Seeds mit beiden Werten
        n = diffs.shape[0]
        if n == 0:
            mean_diff = np.nan
            sd = se = ci_low = ci_high = np.nan
        else:
            mean_diff = diffs.mean()
            sd = diffs.std(ddof=1) if n > 1 else 0.0
            se = sd / np.sqrt(n) if n > 1 else 0.0
            # 95%-CI mit t-Verteilung (bei n=1 kein CI sinnvoll)
            if n > 1:
                t_crit = stats.t.ppf(0.975, df=n-1)
                ci_low = mean_diff - t_crit * se
                ci_high = mean_diff + t_crit * se
            else:
                ci_low = ci_high = np.nan

        results.append({
            "domain": dom,
            "mode": mode,
            "n_seed_pairs": n,
            "mean_improvement_over_base": mean_diff,
            "sd_diff": sd,
            "se_diff": se,
            "ci95_low": ci_low,
            "ci95_high": ci_high,
        })

improvement_table = pd.DataFrame(results).sort_values(["domain", "mode"]).reset_index(drop=True)
print("\n=== Domänen × Modus: Verbesserung ggü. base (gepaart pro Seed) ===")
print(improvement_table)



=== TTA-only: Mode-Effekte ===
              Mixed Linear Model Regression Results
Model:                MixedLM     Dependent Variable:     accuracy
No. Observations:     132         Method:                 ML      
No. Groups:           4           Scale:                  0.0021  
Min. group size:      33          Log-Likelihood:         188.1686
Max. group size:      33          Converged:              Yes     
Mean group size:      33.0                                        
------------------------------------------------------------------
                         Coef. Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------------
Intercept                0.134    0.055  2.450 0.014  0.027  0.241
C(mode)[T.selective_0_1] 0.715    0.019 38.083 0.000  0.678  0.752
C(mode)[T.selective_0_2] 0.694    0.019 36.972 0.000  0.657  0.731
C(mode)[T.selective_0_3] 0.007    0.019  0.382 0.702 -0.030  0.044
C(mode)[T.selective_1_2] 0.692    0.019 36.86

/home/hahlers/miniconda3/envs/server_env/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:1533: RuntimeWarning: divide by zero encountered in log
  cov_aug_logdet = cov_re_logdet + np.sum(np.log(vc_var))
/home/hahlers/miniconda3/envs/server_env/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:1537: RuntimeWarning: divide by zero encountered in divide
  solver = _smw_solver(1., ex_r, ex2_r, cov_re_inv, 1 / vc_var)
/home/hahlers/miniconda3/envs/server_env/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:1542: RuntimeWarning: divide by zero encountered in divide
  ld = _smw_logdet(1., ex_r, ex2_r, cov_re_inv, 1 / vc_var,
/home/hahlers/miniconda3/envs/server_env/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:573: RuntimeWarning: invalid value encountered in scalar add
  return B_logdet + ld + ld1
/home/hahlers/miniconda3/envs/server_env/lib/python3.10/site-packages/statsmodels/regression/m

**paired t-test**

In [9]:
# 1) Daten für MixStyle und single_1
df_pair = df[df["mode"].isin(["base", "single_1"])].copy()

# 2) Mittelwert pro Domäne & Modus (Seeds aggregieren)
df_mean = df_pair.groupby(["domain", "mode"], observed=True)["accuracy"].mean().unstack("mode")

# 3) Paired t-Test (Differenz pro Domäne: single_1 - base)
t_stat, p_val = stats.ttest_rel(df_mean["single_1"], df_mean["base"])

print("=== Paired t-Test: MixStyle (base) vs. TTA (single_1) ===")
print("t-Statistic:", t_stat)
print("p-Value:", p_val)
print("\nMean Accuracy (over seeds):")
print(df_mean)
print("\nMean Difference (single_1 - base):", (df_mean["single_1"] - df_mean["base"]).mean())


=== Paired t-Test: MixStyle (base) vs. TTA (single_1) ===
t-Statistic: 2.100453566566424
p-Value: 0.1265105657848677

Mean Accuracy (over seeds):
mode              base  single_1
domain                          
art painting  0.853779  0.874667
cartoon       0.777022  0.795733
photo         0.969928  0.968633
sketch        0.738481  0.787967

Mean Difference (single_1 - base): 0.021947249330748597
